In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import cv2 as cv 
import glob
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler,Subset
from torchvision import transforms
from IPython.display import HTML
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torchvision  import datasets 
from sklearn.model_selection import KFold
import optuna
from optuna.trial import Trial
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Subset, DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
combined_data = "/kaggle/input/deepfake-and-real-images/Dataset/Test"

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),         # Convert to Tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize
])

In [5]:
combinedData = datasets.ImageFolder(combined_data, transform=transform)

In [6]:
combinedData

Dataset ImageFolder
    Number of datapoints: 10905
    Root location: /kaggle/input/deepfake-and-real-images/Dataset/Test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [7]:
kf = KFold(n_splits=3, shuffle=True, random_state=42) 

In [8]:
samples = len(combinedData)
samples

10905

In [9]:
folds = list(kf.split(np.arange(samples)))

In [10]:
folds

[(array([    1,     2,     4, ..., 10900, 10901, 10902]),
  array([    0,     3,     8, ..., 10899, 10903, 10904])),
 (array([    0,     1,     3, ..., 10902, 10903, 10904]),
  array([    2,    15,    18, ..., 10889, 10900, 10901])),
 (array([    0,     2,     3, ..., 10901, 10903, 10904]),
  array([    1,     4,     5, ..., 10896, 10897, 10902]))]

# Split our data into 3 Kfolds

In [11]:
for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
    print(f"Fold {fold_idx + 1}")
    
    train_size = int(0.75* len(train_val_idx)) 
    val_size = len(train_val_idx) - train_size #25
    
    train_idx = train_val_idx[:train_size]
    val_idx = train_val_idx[train_size:]
    
    # Create subsets
    train_set = Subset(combinedData, train_idx)
    val_set = Subset(combinedData, val_idx)
    test_set = Subset(combinedData, test_idx)
    
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
    
    print(f"Train size: {len(train_set)}")
    print(f"Validation size: {len(val_set)}")
    print(f"Test size: {len(test_set)}")

Fold 1
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 2
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 3
Train size: 5452
Validation size: 1818
Test size: 3635


# Define our model VGG16

In [13]:

# def get_model(name):
#     if name == "vgg16":
#         model = models.vgg16(pretrained=True)
#         model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2) 
#     else:
#         raise ValueError("Model name must be 'vgg16'")
#     return model

In [14]:
# Fine-tune


import torch.nn as nn
from torchvision import models

def get_vgg16_model():
    # Load the pretrained VGG16 model
    model = models.vgg16(pretrained=True)
    
    # Freeze all convolutional layers
    for param in model.features.parameters():
        param.requires_grad = False

    # Unfreeze the last convolutional block (conv5_1, conv5_2, conv5_3)
    for layer in model.features[24:]:  # conv5 layers
        for param in layer.parameters():
            param.requires_grad = True

    # Modify the final fully connected layer
    model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 2)
    
    # Print the number of trainable parameters
    def count_trainable_params(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Total trainable parameters: {count_trainable_params(model):,}")
    
    return model


# Training function

In [68]:
# def TrainingModels(model, train_loader, criterion, optimizer):
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     for images, labels in tqdm(train_loader, desc="Training", leave=False):
#         images, labels = images.to(device), labels.to(device)
        
#         outputs = model(images)
#         loss = criterion(outputs, labels)  #loss function used to compute the error between predictions and true labels
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item() * images.size(0)
#         _, preds = torch.max(outputs, 1) # max score from the scores of two classes
#         correct += (preds == labels).sum().item()
#         total += labels.size(0)

#     epoch_loss = running_loss / total
#     epoch_acc = correct / total
#     return epoch_loss, epoch_acc


In [15]:
# Fine-tune

def TrainingModels(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Add gradient clipping to prevent exploding gradients
    max_grad_norm = 1.0
    
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        
        # Clip gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Eval function

In [16]:
def evaluate(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Optimize the Learning rate using Optuna

In [70]:
# def LR_optimization(trial: Trial):
#     lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    
#     # Iterate over folds
#     fold_results = []
#     for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
#         print(f"\nFold {fold_idx + 1}")
        
#         train_size = int(0.75 * len(train_val_idx))
#         train_idx = train_val_idx[:train_size]
#         val_idx = train_val_idx[train_size:]

#         train_set = Subset(combinedData, train_idx)
#         val_set = Subset(combinedData, val_idx)
#         test_set = Subset(combinedData, test_idx)

#         train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
#         val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
#         test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

        
#         for model_name in ["resnet-50"]:
#             print(f"\nTraining model: {model_name.upper()} on Fold {fold_idx + 1}")
            
#             # Initialize model
#             model = get_model(model_name).to(device)
#             criterion = nn.CrossEntropyLoss()
#             optimizer = optim.Adam(model.parameters(), lr=lr)

#             for epoch in range(5):
#                 current_lr = optimizer.param_groups[0]["lr"]
#                 print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
#                 # Train and evaluate
#                 train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
#                 val_loss, val_acc = evaluate(model, val_loader, criterion)

#                 print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
#                 print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
#             _, val_acc = evaluate(model, val_loader, criterion)
#             fold_results.append(val_acc) 

#     mean_val_acc = sum(fold_results) / len(fold_results)
#     return mean_val_acc



In [17]:
# Fine-tune

def LR_optimization(trial: Trial):
    # Use a smaller learning rate range for fine-tuning
    lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
    
    # Rest of the function remains the same
    fold_results = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"\nFold {fold_idx + 1}")
        
        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

        for model_name in ["vgg16"]:
            print(f"\nFine-tuning model: {model_name.upper()} on Fold {fold_idx + 1}")
            
            model = get_model(model_name).to(device)
            criterion = nn.CrossEntropyLoss()
            
            # Use a smaller weight decay for fine-tuning
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            
            # Add learning rate scheduler for fine-tuning
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='max', factor=0.5, patience=2, verbose=True
            )

            for epoch in range(5):
                current_lr = optimizer.param_groups[0]["lr"]
                print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
                train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
                val_loss, val_acc = evaluate(model, val_loader, criterion)
                
                # Update learning rate based on validation accuracy
                scheduler.step(val_acc)

                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
                print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            _, val_acc = evaluate(model, val_loader, criterion)
            fold_results.append(val_acc)

    mean_val_acc = sum(fold_results) / len(fold_results)
    return mean_val_acc

In [18]:
study = optuna.create_study(direction="maximize")
study.optimize(LR_optimization, n_trials=5)

best_trial = study.best_trial
print("\nBest Trial:")
print(f"  Value (Mean Validation Accuracy): {best_trial.value}")
print(f"  Params: {best_trial.params}")

[I 2024-12-22 18:43:48,463] A new study created in memory with name: no-name-fa65b876-0836-4686-ba74-d93147208da0
<ipython-input-17-5911dd778ed9>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.



Fold 1

Fine-tuning model: VGG16 on Fold 1


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 222MB/s]  
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Epoch 1/5 | Learning Rate: 0.000004


Train Loss: 0.5998, Train Acc: 0.6708
Val Loss: 0.8823, Val Acc: 0.3773

Epoch 2/5 | Learning Rate: 0.000004


...

Train Loss: 0.6667, Train Acc: 0.6711
Val Loss: 0.9866, Val Acc: 0.0000

Epoch 4/5 | Learning Rate: 0.000591


Train Loss: 0.6434, Train Acc: 0.6781
Val Loss: 0.9743, Val Acc: 0.0000

Epoch 5/5 | Learning Rate: 0.000296


Train Loss: 0.6311, Train Acc: 0.6783
Val Loss: 1.0872, Val Acc: 0.0000


[I 2024-12-22 21:15:44,805] Trial 4 finished with value: 0.0 and parameters: {'lr': 0.0005914324576944254}. Best is trial 3 with value: 0.8371837183718371.



Best Trial:
  Value (Mean Validation Accuracy): 0.8371837183718371
  Params: {'lr': 2.5126450824282805e-05}


# After the Optuna optimization process, we will use the best trial to set the learning rate

In [19]:
# Best learning rate from Optuna
best_lr = 2.5126450824282805e-05

In [20]:
final_results = {}

for model_name in ["vgg16"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    # Initialize model
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    
    # Create DataLoaders (assuming combinedData is split into train and test datasets)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    # Train the model
    for epoch in range(10):  # Example: 10 epochs
        print(f"\nEpoch {epoch + 1}/10")
        
        # Train and evaluate
        train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    
    # Final evaluation on the test set
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    
    # Store results
    final_results[model_name] = {
        "Test Loss": test_loss,
        "Test Accuracy": test_acc
    }

# Print final results
print("\nFinal Results:")
for model_name, metrics in final_results.items():
    print(f"{model_name.upper()}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")


Training model: VGG16 with best learning rate 2.5126450824282805e-05

Epoch 1/10


Train Loss: 0.4316, Train Acc: 0.7867

Epoch 2/10


Train Loss: 0.1955, Train Acc: 0.9208

Epoch 3/10


Train Loss: 0.0862, Train Acc: 0.9688

Epoch 4/10


Train Loss: 0.0416, Train Acc: 0.9850

Epoch 5/10


...

Train Loss: 0.0034, Train Acc: 0.9991

Epoch 8/10


Train Loss: 0.0125, Train Acc: 0.9952

Epoch 9/10


Train Loss: 0.0066, Train Acc: 0.9983

Epoch 10/10


Train Loss: 0.0031, Train Acc: 0.9987


Test Loss: 0.7899, Test Acc: 0.9249

Final Results:
VGG16:
  Test Loss: 0.7899
  Test Accuracy: 0.9249


In [21]:
# Function to evaluate the model and get performance metrics
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Initialize cross-validation metrics storage
confusion_matrices = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

best_lr = 2.5126450824282805e-05

for model_name in ["vgg16"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"Fold {fold_idx + 1}")

        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
        
        
        model = get_model(model_name).to(device)
        optimizer = optim.Adam(model.parameters(), lr=best_lr)
        criterion = nn.CrossEntropyLoss()

        # Train the model
        for epoch in range(10):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f"Epoch {epoch+1}/10, Loss: {running_loss / len(train_loader):.4f}")

    
        preds, labels = evaluate_model(model, test_loader)

        
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average='weighted', zero_division=0)
        recall = recall_score(labels, preds, average='weighted', zero_division=0)
        f1 = f1_score(labels, preds, average='weighted', zero_division=0)
        cm = confusion_matrix(labels, preds)

        # Store metrics for the fold
        confusion_matrices.append(cm)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold_idx + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Confusion Matrix:\n{cm}\n")

    # Average metrics across folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1_score = np.mean(f1_scores)
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)

    # Print final average metrics for the model
    print(f"\nFinal Results for {model_name.upper()}:")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1-Score: {avg_f1_score:.4f}")
    print(f"Average Confusion Matrix:\n{avg_confusion_matrix}")


Training model: VGG16 with best learning rate 2.5126450824282805e-05
Fold 1
Epoch 1/10, Loss: 0.4463
Epoch 2/10, Loss: 0.1779
Epoch 3/10, Loss: 0.0874
Epoch 4/10, Loss: 0.0307
Epoch 5/10, Loss: 0.0083
Epoch 6/10, Loss: 0.0110
Epoch 7/10, Loss: 0.0100
Epoch 8/10, Loss: 0.0157
Epoch 9/10, Loss: 0.0017
Epoch 10/10, Loss: 0.0003
Fold 1 - Accuracy: 0.9235, Precision: 0.9247, Recall: 0.9235, F1-Score: 0.9234
Confusion Matrix:
[[1774   90]
 [ 188 1583]]

Fold 2


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.4653
Epoch 2/10, Loss: 0.1844
Epoch 3/10, Loss: 0.0756
Epoch 4/10, Loss: 0.0222
Epoch 5/10, Loss: 0.0183
Epoch 6/10, Loss: 0.0072
Epoch 7/10, Loss: 0.0118
Epoch 8/10, Loss: 0.0094
Epoch 9/10, Loss: 0.0004
Epoch 10/10, Loss: 0.0001
Fold 2 - Accuracy: 0.9221, Precision: 0.9245, Recall: 0.9221, F1-Score: 0.9220
Confusion Matrix:
[[1761   73]
 [ 210 1591]]

Fold 3


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.4179
Epoch 2/10, Loss: 0.1778
Epoch 3/10, Loss: 0.0789
Epoch 4/10, Loss: 0.0197
Epoch 5/10, Loss: 0.0154
Epoch 6/10, Loss: 0.0122
Epoch 7/10, Loss: 0.0033
Epoch 8/10, Loss: 0.0091
Epoch 10/10, Loss: 0.0084
Fold 3 - Accuracy: 0.9202, Precision: 0.9205, Recall: 0.9202, F1-Score: 0.9202
Confusion Matrix:
[[1673  121]
 [ 169 1672]]


Final Results for VGG16:
Average Accuracy: 0.9220
Average Precision: 0.9232
Average Recall: 0.9220
Average F1-Score: 0.9219
Average Confusion Matrix:
[[1736.           94.66666667]
 [ 189.         1615.33333333]]
